# Copyright Netherlands eScience Center and Centrum Wiskunde & Informatica <br>
** Function     : Emotion recognition and forecast with BBConvLSTM** <br>
** Author       : Yang Liu** <br>
** Contributor  : Tianyi Zhang (Centrum Wiskunde & Informatica)<br>
** Last Update  : 2021.02.08 ** <br>
** Last Update  : 2021.02.12 ** <br>
** Library      : Pytorth, Numpy, os, DLACs, matplotlib **<br>
Description     : This notebook serves to test the prediction skill of deep neural networks in emotion recognition and forecast. The Bayesian convolutional Long Short Time Memory neural network with Bernoulli approximate variational inference is used to deal with this spatial-temporal sequence problem. We use Pytorch as the deep learning framework. <br>
<br>
** Many to one prediction.** <br>

Return Values   : Time series and figures <br>

**This project is a joint venture between NLeSC and CWI** <br>

The method comes from the study by Shi et. al. (2015) Convolutional LSTM Network: A Machine Learning Approach for Precipitation Nowcasting. <br>

In [1]:
%matplotlib inline

import sys
import numbers
import pickle

# for data loading
import os
# for pre-processing and machine learning
import numpy as np
import csv
#import sklearn
from scipy.signal import resample

# for visualization
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

The testing device is Dell Inspirion 5680 with Intel Core i7-8700 x64 CPU and Nvidia GTX 1060 6GB GPU.<br>
Here is a benchmark about cpu v.s. gtx 1060 <br>
https://www.analyticsindiamag.com/deep-learning-tensorflow-benchmark-intel-i5-4210u-vs-geforce-nvidia-1060-6gb/

In [2]:
################################################################################# 
#########                           datapath                             ########
#################################################################################
# please specify data path
datapath = 'H:\\Creator_Zone\\Script_craft\\NEmo\\Data_CASE'
output_path = 'H:\\Creator_Zone\\Script_craft\\NEmo\\results'
model_path = 'H:\\Creator_Zone\\Script_craft\\NEmo\\models'
# please specify the constants for input data
window_size = 2000 # down-sampling constant
seq = 20
v_a = 0 # valance = 0, arousal = 1
# leave-one-out training and testing
num_s = 2

In [3]:
f = open(os.path.join(datapath, 'data_{}s'.format(int(window_size/100))),'rb')
data = pickle.load(f)
f.close()
    
samples = data["Samples"]
labels = data["label_s"]
subject_id = data["Subject_id"]

In [4]:
print(subject_id)

[[ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [10]
 [10]
 [10]
 [10]
 [10]
 [10]
 [10]
 [10]
 [11]
 [11]
 [11]
 [11]
 [11]
 [11]
 [11]
 [11]
 [12]
 [12]
 [12]
 [12]
 [12]
 [12]
 [12]
 [12]
 [13]
 [13]
 [13]
 [13]
 [13]
 [13]
 [13]
 [13]
 [14]
 [14]
 [14]
 [14]
 [14]
 [14]
 [14]
 [14]
 [15]
 [15]
 [15]
 [15]
 [15]
 [15]
 [15]
 [15]
 [16]
 [16]
 [16]
 [16]
 [16]
 [16]
 [16]
 [16]
 [17]
 [17]
 [17]
 [17]
 [17]
 [17]
 [17]
 [17]
 [18]
 [18]
 [18]
 [18]
 [18]
 [18]
 [18]
 [18]
 [19]
 [19]
 [19]
 [19]
 [19]
 [19]
 [19]
 [19]
 [20]
 [20]
 [20]
 [20]
 [20]
 [20]
 [20]
 [20]
 [21]
 [21]
 [21]
 [21]
 [21]
 [21]
 [21

In [ ]:
    x_train = samples[np.where(subject_id!=num_s)[0],:,0:4]
    x_test = samples[np.where(subject_id==num_s)[0],:,0:4]
    y_train = np.zeros([0,int(window_size/seq),1])
    y_test = np.zeros([0,int(window_size/seq),1])
    for i in range(len(labels)):
        sig = resample(labels[i][:,v_a],int(window_size/seq)).reshape([1,-1,1])/9
        if subject_id[i] == num_s:
            y_test = np.concatenate([y_test,sig],axis = 0)
        else:
            y_train = np.concatenate([y_train,sig],axis = 0)